In [ ]:
def create_user(conn, login, name):
    llogin = login.lower()
    lock = acquire_lock_with_timeout(conn, f'user:{llogin}', 1)
    
    if not lock:
        return None
    
    if conn.hget('users:', llogin):
        return None

    id = conn.incr('user:id:')
    pipeline = conn.pipeline(True)
    pipeline.hset('user:', llogin, id)
    pipeline.hmset(f'user:{id}', {
        'login': login,
        'id': id,
        'name': name,
        'followers': 0,
        'following': 0,
        'posts': 0,
        'signup': time.time()
    })
    pipeline.execute()
    release_lock(conn, f'user:{llogin}', lock)
    return id